In [36]:
import os
import re
import numpy as np
import collections
import datetime
starttime = datetime.datetime.now()
query_txt = open("query_list.txt")
doc_txt = open("doc_list.txt")
query_list = []
query_temp = []
query = lambda: collections.defaultdict(query)
query_dict=query()
doc = lambda: collections.defaultdict(doc)
doc_dict=doc()
doc_list = []
doc_temp = []
Alldoc = []
Allquery =[]
path_q = "Query"
path_d = "Document"


for line in query_txt.readlines():
    line=line.strip('\n')
    query_list.append(line)
query_txt.close()

for line in doc_txt.readlines():
    line=line.strip('\n')
    doc_list.append(line)
doc_txt.close()

for i in range(len(query_list)):
    query_temp=[]
    dict_temp={}
    f = open(path_q+"/"+query_list[i])
    for line in f.readlines():
        line=line.strip('\n')
        query_temp.append(line.split())
    f.close()
    query_temp=[y for x in query_temp for y in x]
    query_temp=[z for z in query_temp if z!='-1']
    dict_temp=dict_temp.fromkeys(query_temp)
    word_1=list(dict_temp.keys())
    for z in word_1:
        query_dict["q"+str(i)][z]=query_temp.count(z)
    Allquery.append(query_temp)
    
for i in range(len(doc_list)):
    d = open(path_d+"/"+doc_list[i],'r')
    doc_temp=[]
    dict_temp={}
    for line in d.readlines():
        line=line.strip('\n')
        doc_temp.append(line.split())
    d.close()
    doc_temp=[y for x in doc_temp for y in x]
    doc_temp=[z for z in doc_temp if z!='-1']
    dict_temp=dict_temp.fromkeys(doc_temp)
    #doc_dict["d"+str(j)]=dict_temp.fromkeys(doc_temp)
    word_1=list(dict_temp.keys())
    for z in word_1:
        doc_dict["d"+str(i)][z]=doc_temp.count(z)
    Alldoc.append(doc_temp)

dtf_list=[]
qtf_list=[]
qidf_list=[]
didf_list=[]
qw_list=[]
dw_list=[]
sort_sim=[]
max_q=[]
sim=np.zeros((len(query_list),len(doc_list)))
qmax=np.zeros(len(query_list))
#算qtf qidf
for i in range(len(query_list)):

    qtf=np.zeros(len(Allquery[i]))
    qidf=np.zeros(len(Allquery[i]))
    for k,v in enumerate(Allquery[i]):
        qtf[k]=query_dict["q"+str(i)].get(v,0)
        if(qtf[k]>qmax[i]):
            qmax[i]=qtf[k]
        for m in range(len(query_list)):
            temp=query_dict["q"+str(m)].get(v,0)  
            if(temp>0):
                qidf[k]+=1

    qtf_list.append(qtf.tolist())
    qidf_list.append(qidf.tolist())
    
#算dtf,didf 計算異常
for i in range(len(query_list)):
    dtf_list.append([])
    dtf=np.zeros(len(Allquery[i]))
    didf=np.zeros(len(Allquery[i]))
    for j in range(len(doc_list)): 
        for k,v in enumerate(Allquery[i]):
            dtf[k]=doc_dict["d"+str(j)].get(v,0)
            if(dtf[k]>0):
                didf[k]=didf[k]+1
        dtf_list[i].append(dtf.tolist())
    didf_list.append(didf.tolist())
    
#算d q vector
for i in range(len(query_list)):
    for j in range(len(doc_list)):
        dw_list.append([])
        dw_temp=np.zeros(len(Allquery[i]))
        qw_temp=np.zeros(len(Allquery[i]))
        for k in range(len(Allquery[i])):
            if(qidf_list[i][k]!=0):
                qw_temp[k]=(0.5+0.5*qtf_list[i][k]/qmax[i])*np.log(len(query_list)/qidf_list[i][k])
            else:
                qw_temp[k]=0
            if(didf_list[i][k]!=0):
                dw_temp[k]=dtf_list[i][j][k]*np.log(1+(len(doc_list))/didf_list[i][k])
            else:
                dw_temp[k]=0
        dw_list[i].append(dw_temp.tolist())
    qw_list.append(qw_temp.tolist())
                            
fp = open("submission.txt", "w")
fp.write("Query,RetrievedDocuments")
for i in range(len(query_list)):
    for j in range(len(doc_list)):
        Sum_weight=0
        Sum_dw=0
        Sum_qw=0
        for k in range(len(Allquery[i])):
            Sum_weight=Sum_weight+(dw_list[i][j][k]*qw_list[i][k])
            Sum_dw=Sum_dw+np.square(dw_list[i][j][k])
            Sum_qw=Sum_qw+np.square(qw_list[i][k])
        Sum_dw=np.sqrt(Sum_dw)
        Sum_qw=np.sqrt(Sum_qw)
        if(Sum_dw==0 or Sum_qw==0):
            sim[i][j]=0
        else:
            sim[i][j]=Sum_weight/(Sum_dw*Sum_qw)
    sort_index=np.argsort(sim[i])
    sort_index=sort_index.tolist()
    sort_index.reverse()
    fp.write("\n"+query_list[i]+",")
    for k in range(len(doc_list)):
        fp.write(doc_list[sort_index[k]]+" ")
fp.close()


In [50]:
import os
import re
import numpy as np
import collections
import datetime
starttime = datetime.datetime.now()
query_txt = open("query_list.txt")
doc_txt = open("doc_list.txt")
query_list = []
query_temp = []
query = lambda: collections.defaultdict(query)
query_dict=query()
doc = lambda: collections.defaultdict(doc)
doc_dict=doc()
#IDF_d = lambda: collections.defaultdict(IDF_d)
IDF_dict={}
doc_list = []
doc_temp = []
Alldoc = []
Allquery =[]
path_q = "Query"
path_d = "Document"
#best k3=5k1=1.2 b=0.65
#best k3=5 k1=1.4 b=0.75
#best k3=5 k1=1.6 b=0.75
K3=5.0
K1=1.6
b=0.775
for line in query_txt.readlines():
    line=line.strip('\n')
    query_list.append(line)
query_txt.close()

for line in doc_txt.readlines():
    line=line.strip('\n')
    doc_list.append(line)
doc_txt.close()

#算qtf 
for i in range(len(query_list)):
    query_temp=[]
    dict_temp={}
    f = open(path_q+"/"+query_list[i])
    for line in f.readlines():
        line=line.strip('\n')
        query_temp.append(line.split())
    f.close()
    query_temp=[y for x in query_temp for y in x]
    query_temp=[z for z in query_temp if z!='-1']
    dict_temp=dict_temp.fromkeys(query_temp)
    word_1=list(dict_temp.keys())
    for z in word_1:
        query_dict["q"+str(i)][z]=query_temp.count(z)
    Allquery.append(query_temp)
    
sum_temp=0

#算avg doc len 和 dtf idf
for i in range(len(doc_list)):
    d = open(path_d+"/"+doc_list[i],'r')
    doc_temp=[]
    dict_temp={}
    for line in d.readlines():
        line=line.strip('\n')
        doc_temp.append(line.split())
    d.close()
    doc_temp=[y for x in doc_temp for y in x]
    doc_temp=[z for z in doc_temp if z!='-1']
    sum_temp=len(doc_temp)-5+sum_temp
    dict_temp=dict_temp.fromkeys(doc_temp)
    word_1=list(dict_temp.keys())
    for z in word_1:
        doc_dict["d"+str(i)][z]=doc_temp.count(z)
        if(IDF_dict.get(z,0)==0):
            IDF_dict[z]=1
        else:
            IDF_dict[z]=IDF_dict.get(z)+1
    Alldoc.append(doc_temp)
avglen_doc=sum_temp/float(len(doc_list))

dtf_list=[]
qtf_list=[]
qidf_list=[]
didf_list=[]
qw_list=[]
dw_list=[]
sort_sim=[]
max_q=[]
sim=np.zeros((len(query_list),len(doc_list)))
qmax=np.zeros(len(query_list))

#計算相似度且寫入檔案
fp = open("submission.txt", "w")
fp.write("Query,RetrievedDocuments")
for i in range(len(query_list)):
    for j in range(len(doc_list)):
       # dw_list.append([])
        #dw_temp=np.zeros(len(Allquery[i]))
       # qw_temp=np.zeros(len(Allquery[i]))
        for k in range(len(Allquery[i])):
            #if(qidf_list[i][k]!=0):
            dtf_temp=doc_dict["d"+str(j)].get(Allquery[i][k],0)
            qtf_temp=query_dict["q"+str(i)].get(Allquery[i][k],0)
            IDF_temp=IDF_dict.get(Allquery[i][k],0)
            if(dtf_temp!=0 and qtf_temp!=0):
                doc_weight=((K1+1)*dtf_temp)/(K1*((1-b)+b*((len(Alldoc[j])-5)/avglen_doc))+dtf_temp)
                query_weight=((K3+1)*qtf_temp)/(K3+qtf_temp)
                IDF=np.log(1+(len(doc_list)-IDF_temp+0.5)/(IDF_temp+0.5))
                sim[i][j]=sim[i][j]+(doc_weight*query_weight*IDF)
        #print("score:",sim[i][j])
                #print("q:",i)
                #print("word:",Allquery[i][k])
                #print("dtf_temp:",doc_weight)
                #print("qtf_temp:",qtf_temp)
                #print("qtf:",query_weight)
                #print("IDF_temp",IDF)
                
    sort_index=np.argsort(sim[i])
    sort_index=sort_index.tolist()
    sort_index.reverse()
    fp.write("\n"+query_list[i]+",")
    for k in range(len(doc_list)):
        fp.write(doc_list[sort_index[k]]+" ")
        #print(sim[i][j])
fp.close()
endtime = datetime.datetime.now()
print (endtime - starttime).seconds

24
